In [1]:
import os

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
os.chdir('gdrive/My Drive/MSML 604 Project')

Mounted at /content/gdrive/


In [2]:
import torch
import numpy as np
import time
from sklearn.model_selection import train_test_split



In [3]:
def puf_query(c, w):
    n = c.shape[1]
    phi = np.ones(n+1)
    phi[n] = 1
    for i in range(n-1, -1, -1):
        phi[i] = (2*c[0,i]-1)*phi[i+1]

    r = (np.dot(phi, w) > 0)
    return r
    

In [4]:
# Problem Setup
target = 0.99  # The desired prediction rate
n = 64  # number of stages in the PUF

# Initialize the PUF
np.random.seed(int(time.time()))
data = np.loadtxt('weight_diff.txt')
w = np.zeros((n+1, 1))
for i in range(1, n+2):
    randi_offset = np.random.randint(1, len(data)+1)
    w[i-1] = data[randi_offset-1]

# Syntax to query the PUF:
c = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
r = puf_query(c, w)
# you may remove these two lines

In [48]:

# You can use the puf_query function to generate your training dataset
# ADD YOUR DATASET GENERATION CODE HERE
training_size = 10000
training_c = []
training_r = []

for i in range(0,training_size):
  c = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
  r = puf_query(c, w)

  n = c.shape[1]
  phi = np.ones(n+1)
  phi[n] = 1
  for j in range(n-1, -1, -1):
        phi[j] = (2*c[0,j]-1)*phi[j+1]
  training_c.append(phi)
  training_r.append(r)
  if i % 1000 == 0:
    print(i)


training_c = np.array(training_c).reshape(training_size,-1)
training_r = np.array(training_r).reshape(training_size,-1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [44]:
# X_train, X_test, y_train, y_test = train_test_split(training_c, training_r, test_size=0.20, random_state=42)

In [58]:
from sklearn.ensemble import AdaBoostClassifier

w0 = np.zeros((n+1, 1))  # The estimated value of w.
# Try to estimate the value of w here. This section will be timed. You are
# allowed to use the puf_query function here too, but it will count towards
# the training time.
t0 = time.process_time()
# ADD YOUR TRAINING CODE HERE



model = AdaBoostClassifier()
model.fit(training_c, training_r)

# for i in range(1, n+2):
#     randi_offset = np.random.randint(1, len(data)+1)
#     w0[i-1] = data[randi_offset-1]



t1 = time.process_time()
training_time = t1 - t0  # time taken to get w0
print("Training time:", training_time)
print("Training size:", training_size)





/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training time: 0.7891405770001256
Training size: 10000


In [59]:
# Evaluate your result
n_test = 10000
correct = 0
for i in range(1, n_test+1):
    c_test = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
    r = puf_query(c_test, w)
    # r0 = puf_query(c_test, w0)

    n = c_test.shape[1]
    phi_test = np.ones(n+1)
    phi_test[n] = 1
    for j in range(n-1, -1, -1):
          phi_test[j] = (2*c_test[0,j]-1)*phi_test[j+1]
    r0 =  model.predict(phi_test.reshape(1, -1))
    correct += (r==r0)

    if i % 1000 == 0:
      print(i)

success_rate = correct/n_test
print("Success rate:", success_rate)

# If the success rate is less than 99%, a penalty time will be added
# One second is add for each 0.01% below 99%.
effective_training_time = training_time
if success_rate < 0.99:
    effective_training_time = training_time + 10000*(0.99-success_rate)
print("Effective training time:", effective_training_time)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Success rate: [0.9103]
Effective training time: [797.78914058]


In [61]:
from sklearn.ensemble import GradientBoostingClassifier

t0 = time.process_time()


model = GradientBoostingClassifier()
model.fit(training_c, training_r)

t1 = time.process_time()
training_time = t1 - t0  # time taken to get w0
print("Training time:", training_time)
print("Training size:", training_size)



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training time: 3.2414217180000833
Training size: 10000


In [62]:
# Evaluate your result
n_test = 10000
correct = 0
for i in range(1, n_test+1):
    c_test = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
    r = puf_query(c_test, w)
    # r0 = puf_query(c_test, w0)

    n = c_test.shape[1]
    phi_test = np.ones(n+1)
    phi_test[n] = 1
    for j in range(n-1, -1, -1):
          phi_test[j] = (2*c_test[0,j]-1)*phi_test[j+1]
    r0 =  model.predict(phi_test.reshape(1, -1))
    correct += (r==r0)

    if i % 1000 == 0:
      print(i)

success_rate = correct/n_test
print("Success rate:", success_rate)

# If the success rate is less than 99%, a penalty time will be added
# One second is add for each 0.01% below 99%.
effective_training_time = training_time
if success_rate < 0.99:
    effective_training_time = training_time + 10000*(0.99-success_rate)
print("Effective training time:", effective_training_time)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Success rate: [0.8742]
Effective training time: [1161.24142172]


In [63]:
from sklearn.ensemble import RandomForestClassifier

t0 = time.process_time()


model = RandomForestClassifier()
model.fit(training_c, training_r)

t1 = time.process_time()
training_time = t1 - t0  # time taken to get w0
print("Training time:", training_time)
print("Training size:", training_size)



<ipython-input-63-85f7b98facde>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(training_c, training_r)


Training time: 1.916265175000035
Training size: 10000


In [64]:
# Evaluate your result
n_test = 10000
correct = 0
for i in range(1, n_test+1):
    c_test = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
    r = puf_query(c_test, w)
    # r0 = puf_query(c_test, w0)

    n = c_test.shape[1]
    phi_test = np.ones(n+1)
    phi_test[n] = 1
    for j in range(n-1, -1, -1):
          phi_test[j] = (2*c_test[0,j]-1)*phi_test[j+1]
    r0 =  model.predict(phi_test.reshape(1, -1))
    correct += (r==r0)

    if i % 1000 == 0:
      print(i)

success_rate = correct/n_test
print("Success rate:", success_rate)

# If the success rate is less than 99%, a penalty time will be added
# One second is add for each 0.01% below 99%.
effective_training_time = training_time
if success_rate < 0.99:
    effective_training_time = training_time + 10000*(0.99-success_rate)
print("Effective training time:", effective_training_time)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Success rate: [0.8523]
Effective training time: [1378.91626518]


In [65]:
from sklearn.ensemble import BaggingClassifier

t0 = time.process_time()


model = BaggingClassifier()
model.fit(training_c, training_r)

t1 = time.process_time()
training_time = t1 - t0  # time taken to get w0
print("Training time:", training_time)
print("Training size:", training_size)



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training time: 0.894938965999927
Training size: 10000


In [66]:
# Evaluate your result
n_test = 10000
correct = 0
for i in range(1, n_test+1):
    c_test = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
    r = puf_query(c_test, w)
    # r0 = puf_query(c_test, w0)

    n = c_test.shape[1]
    phi_test = np.ones(n+1)
    phi_test[n] = 1
    for j in range(n-1, -1, -1):
          phi_test[j] = (2*c_test[0,j]-1)*phi_test[j+1]
    r0 =  model.predict(phi_test.reshape(1, -1))
    correct += (r==r0)

    if i % 1000 == 0:
      print(i)

success_rate = correct/n_test
print("Success rate:", success_rate)

# If the success rate is less than 99%, a penalty time will be added
# One second is add for each 0.01% below 99%.
effective_training_time = training_time
if success_rate < 0.99:
    effective_training_time = training_time + 10000*(0.99-success_rate)
print("Effective training time:", effective_training_time)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Success rate: [0.7471]
Effective training time: [2429.89493897]
